# **Procesamiento de Lenguaje Natural**
## **MNA - Tecnológico de Monterrey**
## **Algunos ejemplos para extraer el texto de un archivo de audio.**

# **Caso - 1: modelos whisper de OpenAI con HuggingFace**

* #### **Se tienen diferentes variantes de código abierto de whisper de OpenAI con HuggingFace:**

 https://huggingface.co/models?sort=trending&search=openai%2Fwhisper-

* #### **Estos modelos no requieren el uso de la API de OpenAI.**

#### **Se recomienda el uso de GPU**

In [1]:
# %pip install transformers accelerate
# !apt-get install -y ffmpeg   # si trabajas en tu computadora necesitarás FFMPEG para manejo de archivos de audio y música
# %pip install librosa

In [2]:
import torch
from transformers import pipeline
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import librosa   # paquete para audio y música

import warnings
# para filtrar advertencias:
warnings.filterwarnings("ignore")

c:\Users\hille\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Descargamos el archivo de audio:
# !gdown "https://www.gutenberg.org/files/20029/mp3/20029-01.mp3"

In [4]:
# Con librosa podemos obtener información del archivo:
audio_path = "./20029-01.mp3"
audio, sample_rate = librosa.load(audio_path)
print(f"Duración del audio: {len(audio)/sample_rate:.1f} segundos")

Duración del audio: 73.2 segundos


In [5]:
# <Opcional: si deseas escucharlo>
from IPython.display import Audio
Audio(audio_path)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


In [7]:
# Puedes seleccionar algunas de las siguientes versiones de whisper.
# Revisa la documentación de cada uno en HuggingFace.

#model_id="openai/whisper-tiny"    # el más sencillo y ligero, pero menos preciso.
#model_id="openai/whisper-small"
#model_id="openai/whisper-base"
#model="openai/whisper-medium"  # Generalmente el modelo intermedio en cuanto a desempeño y tamaño.
#model_id = "openai/whisper-large-v3"    # mejor modelo
model_id = "openai/whisper-large-v3-turbo"    # una versión ligera de v3

# Cargamos el modelo:
processor = AutoProcessor.from_pretrained(model_id)
modelo = AutoModelForSpeechSeq2Seq.from_pretrained(model_id).to(device)


In [8]:
# Inicializamos el pipeline:
pipe = pipeline("automatic-speech-recognition",
                model=modelo,
                tokenizer=processor.tokenizer,
                feature_extractor=processor.feature_extractor,
                torch_dtype=torch.float32,      #torch.float16 if torch.cuda.is_available() else torch.float32,
                device=device
                )

Device set to use cpu


In [9]:
# Extraemos el texto del audio:
result = pipe(audio,
              return_timestamps=True,   # Para audios mayores a 30 segs.
              generate_kwargs={"language":"es"}  # Lo detecta automático, pero se le puede indicar el idioma.
              )

You have passed language=es, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=es.


In [10]:
result["text"]   # el texto del audio

' Las Fábulas de Esopo Grabado para LibriVox.org por Paulino Fábula número 31 Las ranas y el pantano seco Vivían dos ranas en un bello pantano, Pero llegó el verano y se secó, por lo cual lo abandonaron para buscar otro con agua. Aliaron en su camino un profundo pozo repleto de agua Y al verlo, dijo una rana a la otra Amiga, bajemos las dos a este pozo Pero, ¿y también se secara el agua de este pozo? Repuso la compañera ¿Cómo crees que subiremos entonces? Al tratar de emprender una acción, analiza primero las consecuencias de ella. Fin de la fábula Esta es una grabación del dominio público. Gracias por ver el video.'

In [11]:
result

{'text': ' Las Fábulas de Esopo Grabado para LibriVox.org por Paulino Fábula número 31 Las ranas y el pantano seco Vivían dos ranas en un bello pantano, Pero llegó el verano y se secó, por lo cual lo abandonaron para buscar otro con agua. Aliaron en su camino un profundo pozo repleto de agua Y al verlo, dijo una rana a la otra Amiga, bajemos las dos a este pozo Pero, ¿y también se secara el agua de este pozo? Repuso la compañera ¿Cómo crees que subiremos entonces? Al tratar de emprender una acción, analiza primero las consecuencias de ella. Fin de la fábula Esta es una grabación del dominio público. Gracias por ver el video.',
 'chunks': [{'timestamp': (0.0, 2.84), 'text': ' Las Fábulas de Esopo'},
  {'timestamp': (2.84, 10.12),
   'text': ' Grabado para LibriVox.org por Paulino'},
  {'timestamp': (10.12, 14.44), 'text': ' Fábula número 31'},
  {'timestamp': (14.44, 20.0), 'text': ' Las ranas y el pantano seco'},
  {'timestamp': (20.0, 26.16),
   'text': ' Vivían dos ranas en un bello 

# **Caso - 2: modelo whisper-1 de OpenAI ... requiere uso de la API de OpenAI**

#### **Revisa la documentación del modelo "whisper-1":** https://platform.openai.com/docs/models/whisper-1

In [12]:
# !pip install openai

In [14]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
api_key = os.getenv("OPENAI_API_KEY")
open_ai = OpenAI(api_key=api_key)

if not api_key:
  raise ValueError("API key no encontrada en los secretos")

In [16]:
# Crear cliente con tu clave de API
client = OpenAI(api_key=api_key)

# Ruta del archivo de audio en audio_path.
# Abrimos el archivo en binario y extraemos el texto con whisper-1:
with open(audio_path, "rb") as audio_file:
    transcript = client.audio.transcriptions.create(model="whisper-1",
                                                    file=audio_file,
                                                    language="es"
                                                    )

In [17]:
transcript.text   # en particular observa el formato de la salida del texto.

'Las fábulas de Esopo. Grabado para LibriVox.org por Paulino. Fábula número 31. Las ranas y el pantano seco. Vivían dos ranas en un bello pantano, pero llegó el verano y se secó, por lo cual lo abandonaron para buscar otro con agua. Aliaron en su camino un profundo pozo repleto de agua, y al verlo, dijo una rana a la otra, «Amiga, bajemos las dos a este pozo». «¿Pero y si también se secara el agua de este pozo?», repuso la compañera. «¿Cómo crees que subiremos entonces?» Al tratar de emprender una acción, analiza primero las consecuencias de ella. Fin de la fábula. Esta es una grabación del dominio público.'

# **Caso - 3: otros modelos en HuggingFace**

* #### **Los modelos Wav2Vec2 abarcan una gran variedad de problemáticas, son modelos abiertos de Meta y muchos de ellos los podemos encontrar en HuggingFace.**

* #### **Estos modelos fueron entrenados con datos muestreados a 16 Hz, por lo que se debe verificar que los datos de entrada tengan esta característica.**

In [ ]:
#!pip install transformers librosa

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import librosa
import torch

In [ ]:
from google.colab import userdata
from huggingface_hub import login

In [ ]:
login(userdata.get('miHuggingFace'))

In [ ]:
# Verificamos si hay GPU disponible:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

In [ ]:
audio_path = "./20029-01.mp3"
audio, sample_rate = librosa.load(audio_path,
                                  sr=16000   # los modelos Wav2Vec2 requieren un muestreo (sample rate) de 16Hz.
                                  )
print(f"Duración del audio: {len(audio)/sample_rate:.1f} segundos")

Duración del audio: 73.2 segundos


In [ ]:
# Modelos entrenados específicamente con datos en español.
# Recuerda revisar la documentación de cada uno en HuggingFace.

model_name = "facebook/wav2vec2-large-xlsr-53-spanish"
#model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-spanish"
#model_name="facebook/wav2vec2-base-10k-voxpopuli-ft-es"

# Cargamos el modelo y el procesador:
processor = Wav2Vec2Processor.from_pretrained(model_name)
modelo = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)


In [ ]:
# Convertimos los datos a un tensor de pyTorch con el muestro seleccionado:
input_values = processor(audio,
                         sampling_rate=sample_rate,
                         return_tensors="pt").input_values.to(device)


In [ ]:
# Realizamos las inferencias sin calcular los gradientes para optimizar recursos,
# ya que no estamos haciendo ajuste de parámetros (fine-tuning):
with torch.no_grad():
    logits = modelo(input_values).logits


In [ ]:
# Decodificamos la salida:
predicted_ids = torch.argmax(logits, dim=-1)   # seleccionamos las salidas (ids) de mayor probabilidad.
transcription = processor.decode(predicted_ids[0])   # obtenemos los tokens (palabras) con los ids.

In [ ]:
transcription   # observa que en este modelo la introducción y el cierre del audio se excluyen de manera automática.

'las ranas y el pantano seco vivían dos ranas en un bello pantano pero llegó el verano y se secó por lo cual lo abandonaron para buscar otro con agua aliaron en su camino un profundo poso repleto de agua y al verlo dijo una rana la otra amiga bajemos las dos a este poso pero is también se secara el agua deste pozo repuso la compañera cómo crees que subiremos entonces al tratar de emprender una acción analiza primro las consecuencias de ella fábula'

# **Caso - 4: Google-Translator**

* #### **Esta opción está basada en el traductor de Google y no solo soporta una gran variedad de idiomas, sino también una buena variedad de acentos para un mismo idioma. En particular para el español existen versiones para diversos acentos de países de latinoamérica.**

* #### **Requiere que el archivo de entrada esté en formato de audio WAV.**

* #### **Es gratuita, pero tiene restricciones si hacemos muchas solicitudes en poco tiempo, en cuyo caso se requiere hacer uso de su API y por lo tanto tendrá un costo. Consulta la documentación.**

* #### **Puedes verifica los idiomas y variantes soportadas en la siguiente liga:**

https://cloud.google.com/speech-to-text/docs/speech-to-text-supported-languages?hl=es-419

* #### **Documentación de SpeechRecognition:** https://pypi.org/project/SpeechRecognition/

In [ ]:
!pip install SpeechRecognition pydub

In [ ]:
from pydub import AudioSegment
import speech_recognition as sr
import os

from IPython.display import Markdown, display

In [ ]:
 # Se requiere formato WAV para usar esta solución de Google,
 # por lo que exportamos el mp3 a wav como sigue:

audio_mp3_path = "./20029-01.mp3"   # el audio de entrada.
wav_file = "./tmp.wav"      # el audio de salida.

audio = AudioSegment.from_mp3(audio_mp3_path)
audio.export(wav_file, format="wav")

<_io.BufferedRandom name='./tmp.wav'>

In [ ]:
# Inicializamos el reconocedor:
recognizer = sr.Recognizer()

# Cargamos y analizamos el archivo en formato WAV:
with sr.AudioFile(wav_file) as source:

    audio_data = recognizer.record(source)  # extracción de los datos del audio

    try:
        # Procedemos con la extracción del texto.
        # Estoy usando la pronunciación que tiene para México,
        # pero en dado caso puede cambiarse:
        texto = recognizer.recognize_google(audio_data,
                                            language="es-MX"
                                            )
        display(Markdown(texto))  # para darle cierto formato a la salida

    except sr.UnknownValueError:
        print("No se pudo entender el audio")
    except sr.RequestError as e:
        print(f"No se pudo solicitar resultados al servicio de reconocimiento; {e}")


las fábulas de Esopo grabado para librivox.org por Paulino fábula número 31 las ranas y el Pantano seco vivían dos ranas en un bello pantano Pero llegó el verano y se secó Por lo cual lo abandonaron para buscar otro con agua aliaron en su camino un profundo pozo repleto de agua y al verlo dijo una rana a la otra amiga bajemos las dos a este pozo pero también se secara el agua de este pozo repuso la compañera Cómo crees que subiremos entonces al tratar de emprender una acción analiza primero las consecuencias de ella fin de la fábula Esta es una grabación del dominio público

In [ ]:
texto

'las fábulas de Esopo grabado para librivox.org por Paulino fábula número 31 las ranas y el Pantano seco vivían dos ranas en un bello pantano Pero llegó el verano y se secó Por lo cual lo abandonaron para buscar otro con agua aliaron en su camino un profundo pozo repleto de agua y al verlo dijo una rana a la otra amiga bajemos las dos a este pozo pero también se secara el agua de este pozo repuso la compañera Cómo crees que subiremos entonces al tratar de emprender una acción analiza primero las consecuencias de ella fin de la fábula Esta es una grabación del dominio público'

In [ ]:
# removemos el archivo WAV temporal:
os.remove(wav_file)